In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime as dt

import platform
from matplotlib import font_manager, rc
#matplotlib 한글깨짐 지원
path = "c:/Windows/Fonts/malgun.ttf"
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...')
rc('axes', unicode_minus=False)  

## Read Data

In [2]:
import os
df_train = pd.read_csv(os.path.abspath("../input")+'/X_train.csv', encoding='cp949')
df_test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv', encoding='cp949')
y_train = pd.read_csv(os.path.abspath("../input")+'/y_train.csv').age
IDtest = df_test.custid.unique()

df_train.head()

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0


## 전처리

In [3]:
# 일시

df_train['hour'] = df_train['sales_time']//100
df_train['minute'] = df_train['sales_time']%100
df_train = df_train.rename({'sales_month':'month', 'sales_day':'day'}, axis='columns')
df_train['year'] = 2017

df_train.loc[df_train['month']>12,'year'] = 2018
df_train.loc[df_train['month']>12,'month'] = df_train.month%12

dt = pd.DataFrame(pd.to_datetime(df_train[['year', 'month', 'day', 'hour','minute']]))
dt.columns = ['dt']

df_train = pd.merge(df_train, dt, left_on = df_train.index, right_on=dt.index)

df_test['hour'] = df_test['sales_time']//100
df_test['minute'] = df_test['sales_time']%100
df_test = df_test.rename({'sales_month':'month', 'sales_day':'day'}, axis='columns')
df_test['year'] = 2017

df_test.loc[df_test['month']>12,'year'] = 2018
df_test.loc[df_test['month']>12,'month'] = df_test.month%12

dt_t = pd.DataFrame(pd.to_datetime(df_test[['year', 'month', 'day', 'hour','minute']]))
dt_t.columns = ['dt']

df_test = pd.merge(df_test, dt_t, left_on = df_test.index, right_on=dt_t.index)

del df_train['key_0']
del df_train['year']
del df_train['month']
del df_train['day']
del df_train['hour']
del df_train['minute']
del df_train['sales_time']

del df_test['key_0']
del df_test['year']
del df_test['month']
del df_test['day']
del df_test['hour']
del df_test['minute']
del df_test['sales_time']

In [4]:
# 지역
df_train['city']= df_train['str_nm'].replace(['무역점','본점'],'강남구').replace(
                '천호점','강동구').replace('신촌점','서대문구')
df_test['city']= df_test['str_nm'].replace(['무역점','본점'],'강남구').replace(
                '천호점','강동구').replace('신촌점','서대문구')

In [5]:
# 환불여부
df_train['refund'] = 0
df_train.loc[df_train['tot_amt'] < 0,'refund'] = 1
df_test['refund'] = 0
df_test.loc[df_test['tot_amt'] < 0,'refund'] = 1

In [6]:
# 이름 비슷한거 통일
df_train.part_nm = df_train.part_nm.replace('여성캐쥬얼','여성캐주얼').replace('스포츠캐쥬얼','스포츠캐주얼').replace(
                    '가정용품파트','가정용품').replace('생식품파트','생식품').replace('공산품파트','공산품').replace(
                    '로얄부틱','로얄부띠끄').replace('잡화파트','잡화')
df_test.part_nm = df_test.part_nm.replace('여성캐쥬얼','여성캐주얼').replace('스포츠캐쥬얼','스포츠캐주얼').replace(
                    '가정용품파트','가정용품').replace('생식품파트','생식품').replace('공산품파트','공산품').replace(
                    '로얄부틱','로얄부띠끄').replace('잡화파트','잡화')

# Feature

In [7]:
# 주구매시간, 구매건수(내점일수), 구매주기, 주말방문비율

df = df_train.groupby('custid')['dt'].agg([
    ('주구매시간', lambda x: int(x.dt.hour.value_counts().index[0])),
    ('방문일수',lambda x: x.nunique()),
    ('구매주기', lambda x: int(((x.max() - x.min()).days) / x.nunique())),
    ('주말구매비율', lambda x : ((x.dt.weekday > 4).sum())/(x.count()))
]).reset_index()
df.주구매시간 = df.주구매시간.astype('str')

df_t = df_test.groupby('custid')['dt'].agg([
    ('주구매시간', lambda x: int(x.dt.hour.value_counts().index[0])),
    ('방문일수',lambda x: x.nunique()),
    ('구매주기', lambda x: int(((x.max() - x.min()).days) / x.nunique())),
    ('주말구매비율', lambda x : ((x.dt.weekday > 4).sum())/(x.count()))
]).reset_index()
df_t.주구매시간 = df_t.주구매시간.astype('str')

In [8]:
# 구매추세(총 363일이니까 33일씩 11번)

week_to = df_train.dt.max()
week_trans = []
for i in range(11):
    week_from = week_to + pd.DateOffset(days=-33)
    week_trans.append(df_train.query('@week_from < dt <= @week_to')
                      .groupby('custid')['str_nm']
                      .agg([(f'w{11-i}', 'count')])
                      .reset_index())
    week_to = week_from

f = pd.DataFrame({'custid': df_train.custid.unique()})
for w in week_trans[::-1]:
    f = pd.merge(f, w, how='left')
f = f.fillna(0)
f['구매추세'] = f.apply(lambda x: np.polyfit(range(11), x[1:], 1)[0].round(2), axis=1)
df = df.merge(f.iloc[:,[0,-1]],how='left')


week_to = df_test.dt.max()
week_trans = []
for i in range(11):
    week_from = week_to + pd.DateOffset(days=-33)
    week_trans.append(df_test.query('@week_from < dt <= @week_to')
                      .groupby('custid')['str_nm']
                      .agg([(f'w{11-i}', 'count')])
                      .reset_index())
    week_to = week_from

f_t = pd.DataFrame({'custid': df_test.custid.unique()})
for w in week_trans[::-1]:
    f_t = pd.merge(f_t, w, how='left')
f_t = f_t.fillna(0)
f_t['구매추세'] = f_t.apply(lambda x: np.polyfit(range(11), x[1:], 1)[0].round(2), axis=1)
df_t = df_t.merge(f_t.iloc[:,[0,-1]],how='left')

In [9]:
# 실구매금액 평균

f = df_train.groupby('custid')[['net_amt']].mean().rename(columns={'net_amt':'실구매금액평균'}).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')[['net_amt']].mean().rename(columns={'net_amt':'실구매금액평균'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [10]:
# 실구매금액최대값

f = df_train.groupby('custid')[['net_amt']].max().rename(columns={'net_amt':'실구매금액최대'}).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')[['net_amt']].max().rename(columns={'net_amt':'실구매금액최대'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [11]:
# 실구매금액합

f = df_train.groupby('custid')[['net_amt']].sum().rename(columns={'net_amt':'총실구매금액'}).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')[['net_amt']].sum().rename(columns={'net_amt':'총실구매금액'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [12]:
# 총할인금액

f = df_train.groupby('custid')[['dis_amt']].sum().rename(columns={'dis_amt':'총할인금액'}).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')[['dis_amt']].sum().rename(columns={'dis_amt':'총할인금액'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [13]:
# 상품값 범위

f = df_train.query('tot_amt>0').groupby('custid')['tot_amt'].agg([
    ('최고가상품값',np.max),
    ('최저가상품값',np.min)
]).reset_index()
f['상품값범위'] = f['최고가상품값']-f['최저가상품값']
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

f_t = df_test.query('tot_amt>0').groupby('custid')['tot_amt'].agg([
    ('최고가상품값',np.max),
    ('최저가상품값',np.min)
]).reset_index()
f_t['상품값범위'] = f_t['최고가상품값']-f_t['최저가상품값']
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [14]:
# 실구매값범위

f = df_train.query('net_amt>0').groupby('custid')['net_amt'].agg([
    ('최고가구매값',np.max),
    ('최저가구매값',np.min)
]).reset_index()
f['실구매값범위'] = f['최고가구매값']-f['최저가구매값']
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

f_t = df_test.query('net_amt>0').groupby('custid')['net_amt'].agg([
    ('최고가구매값',np.max),
    ('최저가구매값',np.min)
]).reset_index()
f_t['실구매값범위'] = f_t['최고가구매값']-f_t['최저가구매값']
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [15]:
# 가격 대비 총 힐인 금액 비율

df_train['dis/tot'] = df_train['dis_amt']/df_train['tot_amt']*100
f = df_train.groupby('custid')[['dis/tot']].mean().rename(columns={'dis/tot':'가격대비할인율'}).reset_index()
df = df.merge(f, how='left',on='custid')

df_test['dis/tot'] = df_test['dis_amt']/df_test['tot_amt']*100
f_t = df_test.groupby('custid')[['dis/tot']].mean().rename(columns={'dis/tot':'가격대비할인율'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [16]:
# 구매 대비 환불

f = df_train.groupby('custid')['refund'].agg([
    ('구매건수',np.size),
    ('환불건수',np.sum)
]).reset_index()
f['구매대비환불비'] = f['환불건수']/f['구매건수']
del f['구매건수']
df = df.merge(f, how = 'left', on='custid')

f_t = df_test.groupby('custid')['refund'].agg([
    ('구매건수',np.size),
    ('환불건수',np.sum)
]).reset_index()
f_t['구매대비환불비'] = f_t['환불건수']/f_t['구매건수']
del f_t['구매건수']
df_t = df_t.merge(f_t, how = 'left', on='custid')

In [17]:
# 구매빈도

f = df_train.groupby('custid')['goodcd'].agg([('구매빈도', 'size')]).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')['goodcd'].agg([('구매빈도', 'size')]).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [18]:
# R, F, M 구하기 

df['R'] = pd.qcut(df['구매주기'], q=10, labels=range(10,0,-1)).astype(int)
df['F'] = pd.qcut(df['구매빈도'], q=10, labels=range(1,11)).astype(int)
df['M'] = pd.qcut(df['실구매금액평균'], q=10, labels=range(1,11)).astype(int)

# RFMscore
weights = [3, 2, 5]
df['RFMscore'] = weights[0] * df['R'] + weights[1] * df['F'] + weights[2] * df['M']

df_t['R'] = pd.qcut(df_t['구매주기'], q=10, labels=range(10,0,-1)).astype(int)
df_t['F'] = pd.qcut(df_t['구매빈도'], q=10, labels=range(1,11)).astype(int)
df_t['M'] = pd.qcut(df_t['실구매금액평균'], q=10, labels=range(1,11)).astype(int)

# RFMscore
weights = [3, 2, 5]
df_t['RFMscore'] = weights[0] * df_t['R'] + weights[1] * df_t['F'] + weights[2] * df_t['M']

In [19]:
# 고객등급

df['고객등급'] = pd.qcut(df['RFMscore'], q=6, labels=range(1,7)).astype(int)

df_t['고객등급'] = pd.qcut(df_t['RFMscore'], q=6, labels=range(1,7)).astype(int)

In [20]:
# 총구매금액

f = df_train.groupby('custid')[['tot_amt']].sum().rename(columns={'tot_amt':'총구매금액'}).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')[['tot_amt']].sum().rename(columns={'tot_amt':'총구매금액'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [21]:
# 평균 할부기간

f = df_train.groupby('custid')[['inst_mon']].mean().rename(columns={'inst_mon':'평균할부기간'}).reset_index()
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')[['inst_mon']].mean().rename(columns={'inst_mon':'평균할부기간'}).reset_index()
df_t = df_t.merge(f_t, how='left',on='custid')

In [22]:
# 매장다양성

f = df_train.groupby('custid')['city'].agg([('매장다양성','nunique')]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

f_t = df_test.groupby('custid')['city'].agg([('매장다양성','nunique')]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [23]:
# 실구매액/할부개월

f=df_train.query('net_amt>0')
f['달별구매액'] = f['net_amt'] / f['inst_mon']
f = f.groupby('custid')[['달별구매액']].mean().reset_index()
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

f_t=df_test.query('net_amt>0')
f_t['달별구매액'] = f_t['net_amt'] / f_t['inst_mon']
f_t = f_t.groupby('custid')[['달별구매액']].mean().reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

<ipython-input-23-eda9f2e71967>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['달별구매액'] = f['net_amt'] / f['inst_mon']
<ipython-input-23-eda9f2e71967>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_t['달별구매액'] = f_t['net_amt'] / f_t['inst_mon']


In [24]:
# 소득별연령

df['소득별연령'] = np.nan
df.loc[(df['달별구매액']>137500)&(df['평균할부기간']>2),'소득별연령'] = '부자중년'
df.loc[(df['달별구매액']>137500)&(df['평균할부기간']<=2),'소득별연령'] = '부자고령'
df.loc[(68750<df['달별구매액'])&(df['달별구매액']<=137500)&(df['평균할부기간']>2),'소득별연령'] = '부자청년'
df.loc[(68750<df['달별구매액'])&(df['달별구매액']<=137500)&(df['평균할부기간']<=2),'소득별연령'] = '중년'
df.loc[(42968.75<df['달별구매액'])&(df['달별구매액']<=68750)&(df['평균할부기간']>2),'소득별연령'] = '부자병아리'
df.loc[(42968.75<df['달별구매액'])&(df['달별구매액']<=68750)&(df['평균할부기간']<=2),'소득별연령'] = '청년'
df.loc[(df['달별구매액']<=42968.75)&(df['평균할부기간']>2),'소득별연령'] = '가난병아리'
df.loc[(df['달별구매액']<=42968.75)&(df['평균할부기간']<=2),'소득별연령'] = '병아리'

df_t['소득별연령'] = np.nan
df_t.loc[(df_t['달별구매액']>137500)&(df_t['평균할부기간']>2),'소득별연령'] = '부자중년'
df_t.loc[(df_t['달별구매액']>137500)&(df_t['평균할부기간']<=2),'소득별연령'] = '부자고령'
df_t.loc[(68750<df_t['달별구매액'])&(df_t['달별구매액']<=137500)&(df_t['평균할부기간']>2),'소득별연령'] = '부자청년'
df_t.loc[(68750<df_t['달별구매액'])&(df_t['달별구매액']<=137500)&(df_t['평균할부기간']<=2),'소득별연령'] = '중년'
df_t.loc[(42968.75<df_t['달별구매액'])&(df_t['달별구매액']<=68750)&(df_t['평균할부기간']>2),'소득별연령'] = '부자병아리'
df_t.loc[(42968.75<df_t['달별구매액'])&(df_t['달별구매액']<=68750)&(df_t['평균할부기간']<=2),'소득별연령'] = '청년'
df_t.loc[(df_t['달별구매액']<=42968.75)&(df_t['평균할부기간']>2),'소득별연령'] = '가난병아리'
df_t.loc[(df_t['달별구매액']<=42968.75)&(df_t['평균할부기간']<=2),'소득별연령'] = '병아리'

In [25]:
# 할인고객

df['할인고객']=np.nan
df.loc[(df['가격대비할인율']<2)&(df['고객등급']>4),'할인고객'] = '베리굿'
df.loc[(df['가격대비할인율']<2)&(df['고객등급']<5)&(df['고객등급']>2),'할인고객']='쏘쏘굿'
df.loc[(df['가격대비할인율']<2)&(df['고객등급']<3),'할인고객'] = '낫굿'

df.loc[(df['가격대비할인율']<3)&(df['가격대비할인율']>=2)&(df['고객등급']>4),'할인고객'] = '쏘쏘굿'
df.loc[(df['가격대비할인율']<3)&(df['가격대비할인율']>=2)&(df['고객등급']<5)&(df['고객등급']>2),'할인고객'] = '낫굿'
df.loc[(df['가격대비할인율']<3)&(df['가격대비할인율']>=2)&(df['고객등급']<3),'할인고객'] = '쏘쏘쏘쏘'

df.loc[(df['가격대비할인율']<4)&(df['가격대비할인율']>=3)&(df['고객등급']>4),'할인고객'] = '베리쏘쏘'
df.loc[(df['가격대비할인율']<4)&(df['가격대비할인율']>=3)&(df['고객등급']<5)&(df['고객등급']>2),'할인고객'] = '쏘쏘쏘쏘'
df.loc[(df['가격대비할인율']<4)&(df['가격대비할인율']>=3)&(df['고객등급']<3),'할인고객'] = '낫쏘쏘'

df.loc[(df['가격대비할인율']<5)&(df['가격대비할인율']>=4)&(df['고객등급']>4),'할인고객'] = '낫쏘쏘'
df.loc[(df['가격대비할인율']<5)&(df['가격대비할인율']>=4)&(df['고객등급']<5)&(df['고객등급']>2),'할인고객'] = '쏘쏘배드'
df.loc[(df['가격대비할인율']<5)&(df['가격대비할인율']>=4)&(df['고객등급']<3),'할인고객'] = '베리배드'

df.loc[(df['가격대비할인율']>=5)&(df['고객등급']>4),'할인고객'] = '낫배드'
df.loc[(df['가격대비할인율']>=5)&(df['고객등급']<5)&(df['고객등급']>2),'할인고객'] = '쏘쏘배드'
df.loc[(df['가격대비할인율']>=5)&(df['고객등급']<3),'할인고객'] = '배리베드'


df_t['할인고객']=np.nan
df_t.loc[(df_t['가격대비할인율']<2)&(df_t['고객등급']>4),'할인고객'] = '베리굿'
df_t.loc[(df_t['가격대비할인율']<2)&(df_t['고객등급']<5)&(df_t['고객등급']>2),'할인고객']='쏘쏘굿'
df_t.loc[(df_t['가격대비할인율']<2)&(df_t['고객등급']<3),'할인고객'] = '낫굿'

df_t.loc[(df_t['가격대비할인율']<3)&(df_t['가격대비할인율']>=2)&(df_t['고객등급']>4),'할인고객'] = '쏘쏘굿'
df_t.loc[(df_t['가격대비할인율']<3)&(df_t['가격대비할인율']>=2)&(df_t['고객등급']<5)&(df_t['고객등급']>2),'할인고객'] = '낫굿'
df_t.loc[(df_t['가격대비할인율']<3)&(df_t['가격대비할인율']>=2)&(df_t['고객등급']<3),'할인고객'] = '쏘쏘쏘쏘'

df_t.loc[(df_t['가격대비할인율']<4)&(df_t['가격대비할인율']>=3)&(df_t['고객등급']>4),'할인고객'] = '베리쏘쏘'
df_t.loc[(df_t['가격대비할인율']<4)&(df_t['가격대비할인율']>=3)&(df_t['고객등급']<5)&(df_t['고객등급']>2),'할인고객'] = '쏘쏘쏘쏘'
df_t.loc[(df_t['가격대비할인율']<4)&(df_t['가격대비할인율']>=3)&(df_t['고객등급']<3),'할인고객'] = '낫쏘쏘'

df_t.loc[(df_t['가격대비할인율']<5)&(df_t['가격대비할인율']>=4)&(df_t['고객등급']>4),'할인고객'] = '낫쏘쏘'
df_t.loc[(df_t['가격대비할인율']<5)&(df_t['가격대비할인율']>=4)&(df_t['고객등급']<5)&(df_t['고객등급']>2),'할인고객'] = '쏘쏘배드'
df_t.loc[(df_t['가격대비할인율']<5)&(df_t['가격대비할인율']>=4)&(df_t['고객등급']<3),'할인고객'] = '베리배드'

df_t.loc[(df_t['가격대비할인율']>=5)&(df_t['고객등급']>4),'할인고객'] = '낫배드'
df_t.loc[(df_t['가격대비할인율']>=5)&(df_t['고객등급']<5)&(df_t['고객등급']>2),'할인고객'] = '쏘쏘배드'
df_t.loc[(df_t['가격대비할인율']>=5)&(df_t['고객등급']<3),'할인고객'] = '배리베드'

In [26]:
# 주기와 실구매금액평균 별 구분

df['사람']=np.nan
df.loc[(df['구매주기']<df['구매주기'].mean())&(df['실구매금액평균']<df['실구매금액평균'].mean()) ,'사람']='자주_적은금액'
df.loc[(df['구매주기']<df['구매주기'].mean())&(df['실구매금액평균']>=df['실구매금액평균'].mean()) ,'사람']='자주_많은금액'
df.loc[(df['구매주기']>=df['구매주기'].mean())&(df['실구매금액평균']<df['실구매금액평균'].mean()) ,'사람']='드물게_적은금액'
df.loc[(df['구매주기']>=df['구매주기'].mean())&(df['실구매금액평균']>=df['실구매금액평균'].mean()) ,'사람']='드물게_많은금액'

df_t['사람']=np.nan
df_t.loc[(df_t['구매주기']<df_t['구매주기'].mean())&(df_t['실구매금액평균']<df_t['실구매금액평균'].mean()) ,'사람']='자주_적은금액'
df_t.loc[(df_t['구매주기']<df_t['구매주기'].mean())&(df_t['실구매금액평균']>=df_t['실구매금액평균'].mean()) ,'사람']='자주_많은금액'
df_t.loc[(df_t['구매주기']>=df_t['구매주기'].mean())&(df_t['실구매금액평균']<df_t['실구매금액평균'].mean()) ,'사람']='드물게_적은금액'
df_t.loc[(df_t['구매주기']>=df_t['구매주기'].mean())&(df_t['실구매금액평균']>=df_t['실구매금액평균'].mean()) ,'사람']='드물게_많은금액'

In [27]:
# 실구매금액최대로 연령 추측

df['연령1'] = np.nan
df.loc[(df['실구매금액최대']<=196750), '연령1'] = '20대'
df.loc[(196750<df['실구매금액최대'])&(df['실구매금액최대']<=221343.75), '연령1'] = '60대'
df.loc[(221343.75<df['실구매금액최대'])&(df['실구매금액최대']<=288976.5625), '연령1'] = '50대'
df.loc[(288976.5625<df['실구매금액최대'])&(df['실구매금액최대']<=750000), '연령1'] = '30대'
df.loc[(750000<df['실구매금액최대']), '연령1'] = '40대'

df_t['연령1'] = np.nan
df_t.loc[(df_t['실구매금액최대']<=196750), '연령1'] = '20대'
df_t.loc[(196750<df_t['실구매금액최대'])&(df_t['실구매금액최대']<=221343.75), '연령1'] = '60대'
df_t.loc[(221343.75<df_t['실구매금액최대'])&(df_t['실구매금액최대']<=288976.5625), '연령1'] = '50대'
df_t.loc[(288976.5625<df_t['실구매금액최대'])&(df_t['실구매금액최대']<=750000), '연령1'] = '30대'
df_t.loc[(750000<df_t['실구매금액최대']), '연령1'] = '40대'

In [28]:
# 수입품 여부

f = df_train.groupby('custid')['import_flg'].agg(['sum','mean']).reset_index().rename(columns = {'sum':'수입여부합','mean':'수입여부평균'})
df = df.merge(f, how='left',on='custid')

f_t = df_test.groupby('custid')['import_flg'].agg(['sum','mean']).reset_index().rename(columns = {'sum':'수입여부합','mean':'수입여부평균'})
df_t = df_t.merge(f_t, how='left',on='custid')

In [29]:
# 전체 인기상품

인기상품 = list(df_train.groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [30]:
# 월요일 인기상품

인기상품 = list(df_train.query('sales_dayofweek=="월"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['월인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['월인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="월"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['월인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['월인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [31]:
# 화요일 인기상품

인기상품 = list(df_train.query('sales_dayofweek=="화"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['화인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['화인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="화"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['화인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['화인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [32]:
# 수요일 인기상품

인기상품 = list(df_train.query('sales_dayofweek=="수"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['수인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['수인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="수"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['수인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['수인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [33]:
# 목요일 인기상품

인기상품 = list(df_train.query('sales_dayofweek=="목"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['목인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['목인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="목"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['목인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['목인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [34]:
# 금요일 인기수치

인기상품 = list(df_train.query('sales_dayofweek=="금"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['금인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['금인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="금"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['금인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['금인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [35]:
# 토요일 인기수치

인기상품 = list(df_train.query('sales_dayofweek=="토"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['토인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['토인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="토"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['토인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['토인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [36]:
# 일요일 인기수치

인기상품 = list(df_train.query('sales_dayofweek=="일"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_train.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['일인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['일인기수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

인기상품 = list(df_test.query('sales_dayofweek=="일"').groupby('pc_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기상품가중치 = [round(i*0.01,2) for i in range(1, len(인기상품)+1)]
인기1 = dict(zip(인기상품, 인기상품가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기상품','인기상품가중치'])

a = df_test.groupby(['custid','pc_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='pc_nm',right_on='인기상품')
a['일인기수치'] = a['part_nm']*a['인기상품가중치']

aa = a.groupby('custid')[['일인기수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

In [37]:
# 인기브랜드

인기브랜드 = list(df_train.groupby('brd_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기브랜드가중치 = [round(i*0.01,2) for i in range(1, len(인기브랜드)+1)]
인기1 = dict(zip(인기브랜드, 인기브랜드가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기브랜드','인기브랜드가중치'])

a = df_train.groupby(['custid','brd_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='brd_nm',right_on='인기브랜드')
a['브랜드수치'] = a['part_nm']*a['인기브랜드가중치']

aa = a.groupby('custid')[['브랜드수치']].mean().reset_index()
df = df.merge(aa, how='left',on='custid')

df_train = df_train.merge(인기, how='left',left_on='brd_nm',right_on='인기브랜드')
df_train['환불수치'] = df_train['인기브랜드가중치']*df_train['refund']*(-2)
del df_train['인기브랜드가중치']

aaa = df_train.groupby('custid')[['환불수치']].mean().reset_index()
df = df.merge(aaa, how='left',on='custid')
del df_train['환불수치']
del df_train['인기브랜드']

df['브랜드'] = df['브랜드수치']+df['환불수치']
del df['브랜드수치']
del df['환불수치']


인기브랜드 = list(df_test.groupby('brd_nm')[['part_nm']].count().sort_values(by='part_nm', ascending=True).index)
인기브랜드가중치 = [round(i*0.01,2) for i in range(1, len(인기브랜드)+1)]
인기1 = dict(zip(인기브랜드, 인기브랜드가중치))
인기 = pd.DataFrame(list(인기1.items()),columns = ['인기브랜드','인기브랜드가중치'])

a = df_test.groupby(['custid','brd_nm'])[['part_nm']].count().reset_index()
a = a.merge(인기, how='left',left_on='brd_nm',right_on='인기브랜드')
a['브랜드수치'] = a['part_nm']*a['인기브랜드가중치']

aa = a.groupby('custid')[['브랜드수치']].mean().reset_index()
df_t = df_t.merge(aa, how='left',on='custid')

df_test = df_test.merge(인기, how='left',left_on='brd_nm',right_on='인기브랜드')
df_test['환불수치'] = df_test['인기브랜드가중치']*df_test['refund']*(-2)
del df_test['인기브랜드가중치']

aaa = df_test.groupby('custid')[['환불수치']].mean().reset_index()
df_t = df_t.merge(aaa, how='left',on='custid')
del df_test['환불수치']
del df_test['인기브랜드']

df_t['브랜드'] = df_t['브랜드수치']+df_t['환불수치']
del df_t['브랜드수치']
del df_t['환불수치']

In [38]:
# 주상품군

f = df_train.groupby('custid')['pc_nm'].agg([
    ('주상품군', lambda x: x.value_counts().index[0])
]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.groupby('custid')['pc_nm'].agg([
    ('주상품군', lambda x: x.value_counts().index[0])
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')

In [39]:
# 완전대분류

df_train['완전대분류'] = df_train['part_nm']
df_train['완전대분류'] = df_train['완전대분류'].replace(['명품잡화','잡화','패션잡화','인터넷백화점','로얄부띠끄'],'패션잡화').replace([
                        '남성의류','남성정장스포츠'],'남성의류').replace(['여성캐주얼','여성의류파트','케주얼,구두,아동','여성정장'],
                        '여성의류').replace(['가정용품','공산품'],'리빙').replace(['영어덜트캐쥬얼','영라이브','영캐릭터','영플라자'],
                        '영패션').replace(['아동','아동문화','아동,스포츠'],'아동').replace(['골프/유니캐쥬얼','스포츠캐주얼'],'스포츠')

df_test['완전대분류'] = df_test['part_nm']
df_test['완전대분류'] = df_test['완전대분류'].replace(['명품잡화','잡화','패션잡화','인터넷백화점','로얄부띠끄'],'패션잡화').replace([
                        '남성의류','남성정장스포츠'],'남성의류').replace(['여성캐주얼','여성의류파트','케주얼,구두,아동','여성정장'],
                        '여성의류').replace(['가정용품','공산품'],'리빙').replace(['영어덜트캐쥬얼','영라이브','영캐릭터','영플라자'],
                        '영패션').replace(['아동','아동문화','아동,스포츠'],'아동').replace(['골프/유니캐쥬얼','스포츠캐주얼'],'스포츠')

In [40]:
# 주구매품목_part(part_nm)

f = df_train.groupby('custid')['part_nm'].agg([('주구매품목part', lambda x: (x.value_counts().index[0]))]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how = 'left', on='custid')

f_t = df_test.groupby('custid')['part_nm'].agg([('주구매품목part', lambda x: (x.value_counts().index[0]))]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how = 'left', on='custid')

In [41]:
# 주구매품목_완전(완전대분류)

f = df_train.groupby('custid')['완전대분류'].agg([('주구매품목완전', lambda x: (x.value_counts().index[0]))]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how = 'left', on='custid')

f_t = df_test.groupby('custid')['완전대분류'].agg([('주구매품목완전', lambda x: (x.value_counts().index[0]))]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how = 'left', on='custid')

In [42]:
# 주구매브랜드(brd_nm)

f = df_train.groupby('custid')['brd_nm'].agg([('주구매브랜드', lambda x: (x.value_counts().index[0]))]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how = 'left', on='custid')

f_t = df_test.groupby('custid')['brd_nm'].agg([('주구매브랜드', lambda x: (x.value_counts().index[0]))]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how = 'left', on='custid')

In [43]:
# 주구매품목(buyer_nm)

f = df_train.groupby('custid')['buyer_nm'].agg([('주구매품목buy', lambda x: (x.value_counts().index[0]))]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how = 'left', on='custid')

f_t = df_test.groupby('custid')['buyer_nm'].agg([('주구매품목buy', lambda x: (x.value_counts().index[0]))]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how = 'left', on='custid')

In [44]:
# 상품구매율

np.percentile(df_train.query('33620000>tot_amt>0').tot_amt, [0,20,40,60,80,100], interpolation='nearest')
df_train['가격']=np.nan
df_train.loc[df_train['tot_amt']<=23500, '가격'] = '완전낮은금액'
df_train.loc[(df_train['tot_amt']>23500)&(df_train['tot_amt']<=44000), '가격'] = '낮은금액'
df_train.loc[(df_train['tot_amt']>44000)&(df_train['tot_amt']<=78000), '가격'] = '보통금액'
df_train.loc[(df_train['tot_amt']>78000)&(df_train['tot_amt']<=152000), '가격'] = '높은금액'
df_train.loc[(df_train['tot_amt']>152000), '가격'] = '완전높은금액'

a = df_train.query('가격=="완전낮은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_train.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['완전낮은금액비율'] = a['part_nm']/a['goodcd']
df = df.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_train.query('가격=="낮은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_train.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['낮은금액비율'] = a['part_nm']/a['goodcd']
df = df.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_train.query('가격=="보통금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_train.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['보통금액비율'] = a['part_nm']/a['goodcd']
df = df.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_train.query('가격=="높은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_train.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['높은금액비율'] = a['part_nm']/a['goodcd']
df = df.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_train.query('가격=="완전높은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_train.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['완전높은금액비율'] = a['part_nm']/a['goodcd']
df = df.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')


np.percentile(df_test.query('tot_amt>0').tot_amt, [0,20,40,60,80,100], interpolation='nearest')
df_test['가격']=np.nan
df_test.loc[df_test['tot_amt']<=24000, '가격'] = '완전낮은금액'
df_test.loc[(df_test['tot_amt']>24000)&(df_test['tot_amt']<=45000), '가격'] = '낮은금액'
df_test.loc[(df_test['tot_amt']>45000)&(df_test['tot_amt']<=79000), '가격'] = '보통금액'
df_test.loc[(df_test['tot_amt']>79000)&(df_test['tot_amt']<=155000), '가격'] = '높은금액'
df_test.loc[(df_test['tot_amt']>155000), '가격'] = '완전높은금액'

a = df_test.query('가격=="완전낮은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_test.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['완전낮은금액비율'] = a['part_nm']/a['goodcd']
df_t = df_t.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_test.query('가격=="낮은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_test.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['낮은금액비율'] = a['part_nm']/a['goodcd']
df_t = df_t.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_test.query('가격=="보통금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_test.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['보통금액비율'] = a['part_nm']/a['goodcd']
df_t = df_t.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_test.query('가격=="높은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_test.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['높은금액비율'] = a['part_nm']/a['goodcd']
df_t = df_t.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

a = df_test.query('가격=="완전높은금액"').groupby('custid')[['part_nm']].count().reset_index()
count = df_test.groupby('custid')[['goodcd']].count().reset_index()
a = a.merge(count, how = 'left', on = 'custid')
a['완전높은금액비율'] = a['part_nm']/a['goodcd']
df_t = df_t.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')

df = df.fillna(0)
df_t = df_t.fillna(0)

In [45]:
# 주 구매월

f = df_train.groupby('custid')['dt'].agg([
    ('주구매월', lambda x: int(x.dt.month.value_counts().index[0]))
]).reset_index()
df = df.merge(f, how='left', on='custid')
df.주구매월 = df.주구매월.astype('str')

f_t = df_test.groupby('custid')['dt'].agg([
    ('주구매월', lambda x: int(x.dt.month.value_counts().index[0]))
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')
df_t.주구매월 = df_t.주구매월.astype('str')

In [46]:
# 주구매요일

f = df_train.groupby('custid')['sales_dayofweek'].agg([
    ('주구매요일', lambda x: x.value_counts().index[0])
]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.groupby('custid')['sales_dayofweek'].agg([
    ('주구매요일', lambda x: x.value_counts().index[0])
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')

In [47]:
# 행사상품구매건수

f = df_train.query('dis_amt>0').groupby('custid')['goodcd'].agg([
    ('행사건수','count')]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.query('dis_amt>0').groupby('custid')['goodcd'].agg([
    ('행사건수','count')]).reset_index()
df_t = df_t.merge(f, how='left', on='custid')

df = df.fillna(0)
df_t = df_t.fillna(0)

In [48]:
# 주 환불상품

f = df_train.query('tot_amt<0').groupby('custid')['pc_nm'].agg([
        ('주환불상품', lambda x: x.value_counts().index[0])
]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.query('tot_amt<0').groupby('custid')['pc_nm'].agg([
        ('주환불상품', lambda x: x.value_counts().index[0])
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')

df = df.fillna('없음')
df_t = df_t.fillna('없음')

In [49]:
# 중복구매 브랜드

f = df_train.query('tot_amt>0').groupby('custid')['brd_nm'].agg([
    ('중복구매브랜드', lambda x: x.value_counts().index[0])
]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.query('tot_amt>0').groupby('custid')['brd_nm'].agg([
    ('중복구매브랜드', lambda x: x.value_counts().index[0])
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')

In [50]:
# 중복구매상품

f = df_train.query('tot_amt>0').groupby('custid')['pc_nm'].agg([
    ('중복구매상품', lambda x: x.value_counts().index[0])
]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.query('tot_amt>0').groupby('custid')['pc_nm'].agg([
    ('중복구매상품', lambda x: x.value_counts().index[0])
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')

In [51]:
# 겹치는 가중치

a = df[['custid','주상품군','중복구매상품','주환불상품']]

a[['주상품중복동일']] = np.nan
a.loc[a['주상품군']==a['중복구매상품'],'주상품중복동일']=0.1

a[['주상품환불동일']] = np.nan
a.loc[a['주상품군']==a['주환불상품'],'주상품환불동일'] = 0.2

a[['중복환불동일']] = np.nan
a.loc[a['중복구매상품']==a['주환불상품'],'중복환불동일'] = 0.3
a=a.fillna(0)

a['겹치는가중치'] = a['주상품중복동일']+a['주상품환불동일']+a['중복환불동일']
df = df.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')
df['겹치는가중치'] = df['겹치는가중치'].round(5)


a = df_t[['custid','주상품군','중복구매상품','주환불상품']]

a[['주상품중복동일']] = np.nan
a.loc[a['주상품군']==a['중복구매상품'],'주상품중복동일']=0.1

a[['주상품환불동일']] = np.nan
a.loc[a['주상품군']==a['주환불상품'],'주상품환불동일'] = 0.2

a[['중복환불동일']] = np.nan
a.loc[a['중복구매상품']==a['주환불상품'],'중복환불동일'] = 0.3
a=a.fillna(0)

a['겹치는가중치'] = a['주상품중복동일']+a['주상품환불동일']+a['중복환불동일']
df_t = df_t.merge(a.iloc[:,[0,-1]], how = 'left', on='custid')
df_t['겹치는가중치'] = df_t['겹치는가중치'].round(5)

<ipython-input-51-ad4035c94884>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a[['주상품중복동일']] = np.nan
C:\Users\rudtj\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-51-ad4035c94884>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

In [52]:
# 선호계절

df_train['계절'] = df_train['dt'].dt.month
df_train['계절'] = df_train[['계절']].replace([3,4,5],'봄').replace([6,7,8],'여름').replace([9,10,11],'가을').replace([12,1,2],'겨울')
f = df_train.groupby('custid')['계절'].agg([('선호계절',lambda x : (x.value_counts().index[0]))]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

df_test['계절'] = df_test['dt'].dt.month
df_test['계절'] = df_test[['계절']].replace([3,4,5],'봄').replace([6,7,8],'여름').replace([9,10,11],'가을').replace([12,1,2],'겨울')
f_t= df_test.groupby('custid')['계절'].agg([('선호계절',lambda x : (x.value_counts().index[0]))]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [53]:
# 분기

df_train['분기'] = df_train['dt'].dt.month
df_train['분기'] = df_train['분기'].replace([1,2,3],'1분기').replace([4,5,6],'2분기').replace([7,8,9],'3분기').replace([10,11,12],'4분기')
f = df_train.groupby('custid')['분기'].agg([('분기',lambda x : (x.value_counts().index[0]))]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

df_test['분기'] = df_test['dt'].dt.month
df_test['분기'] = df_test['분기'].replace([1,2,3],'1분기').replace([4,5,6],'2분기').replace([7,8,9],'3분기').replace([10,11,12],'4분기')
f_t = df_test.groupby('custid')['분기'].agg([('분기',lambda x : (x.value_counts().index[0]))]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [54]:
# 주구매지점

f = df_train.query('tot_amt>0').groupby('custid')['city'].agg([
    ('주구매지점', lambda x: x.value_counts().index[0])
]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.query('tot_amt>0').groupby('custid')['city'].agg([
    ('주구매지점', lambda x: x.value_counts().index[0])
]).reset_index()
df_t = df_t.merge(f_t, how='left', on='custid')

In [55]:
# 인기일 가중치

인기요일 = list(df.groupby('주구매요일')[['분기']].count().sort_values(by='분기').index)
인기요일가중치 =  [round(i*(0.1)+0.3,2) for i in range(1, len(인기요일)+1)]
인기요 = dict(zip(인기요일, 인기요일가중치))
인기일 = pd.DataFrame(list(인기요.items()),columns = ['인기요일','인기요일가중치'])

인기계절 = list(df.groupby('선호계절')[['분기']].count().sort_values(by='분기').index)
인기계절가중치 =  [round(i*(0.1)+0.1,2) for i in range(1, len(인기계절)+1)]
인기계 = dict(zip(인기계절, 인기계절가중치))
인기절 = pd.DataFrame(list(인기계.items()),columns = ['인기계절','인기계절가중치'])

인기분기 = list(df.groupby('분기')[['선호계절']].count().sort_values(by='선호계절').index)
인기분기가중치 =  [round(i*(0.1)+0.1,2) for i in range(1, len(인기분기)+1)]
인기분 = dict(zip(인기분기, 인기분기가중치))
인기기 = pd.DataFrame(list(인기분.items()),columns = ['인기분기','인기분기가중치'])

aa = df[['custid','주구매요일','선호계절','분기']]
aa = aa.merge(인기일, how='left',left_on='주구매요일', right_on='인기요일').merge(인기절, how='left',left_on='선호계절', right_on='인기계절').merge(인기기, how='left',left_on='분기', right_on='인기분기')
aa['인기일가중치'] = aa['인기요일가중치']+aa['인기계절가중치']+aa['인기분기가중치']
df = df.merge(aa.iloc[:,[0,-1]], how='left',on='custid')



인기요일 = list(df_t.groupby('주구매요일')[['분기']].count().sort_values(by='분기').index)
인기요일가중치 =  [round(i*(0.1)+0.3,2) for i in range(1, len(인기요일)+1)]
인기요 = dict(zip(인기요일, 인기요일가중치))
인기일 = pd.DataFrame(list(인기요.items()),columns = ['인기요일','인기요일가중치'])

인기계절 = list(df_t.groupby('선호계절')[['분기']].count().sort_values(by='분기').index)
인기계절가중치 =  [round(i*(0.1)+0.1,2) for i in range(1, len(인기계절)+1)]
인기계 = dict(zip(인기계절, 인기계절가중치))
인기절 = pd.DataFrame(list(인기계.items()),columns = ['인기계절','인기계절가중치'])

인기분기 = list(df_t.groupby('분기')[['선호계절']].count().sort_values(by='선호계절').index)
인기분기가중치 =  [round(i*(0.1)+0.1,2) for i in range(1, len(인기분기)+1)]
인기분 = dict(zip(인기분기, 인기분기가중치))
인기기 = pd.DataFrame(list(인기분.items()),columns = ['인기분기','인기분기가중치'])

aa = df_t[['custid','주구매요일','선호계절','분기']]
aa = aa.merge(인기일, how='left',left_on='주구매요일', right_on='인기요일').merge(인기절, how='left',left_on='선호계절', right_on='인기계절').merge(인기기, how='left',left_on='분기', right_on='인기분기')
aa['인기일가중치'] = aa['인기요일가중치']+aa['인기계절가중치']+aa['인기분기가중치']
df_t = df_t.merge(aa.iloc[:,[0,-1]], how='left',on='custid')

In [56]:
(df_t['인기일가중치']+df_t['겹치는가중치']).min(), (df['인기일가중치']+df['겹치는가중치']).max()

(0.9, 2.6)

In [57]:
#트랜드 여부

df['트랜드'] = np.nan
df.loc[(df['인기일가중치']+df['겹치는가중치']<1.25), '트랜드'] = '베리트랜디'
df.loc[(1.25<=df['인기일가중치']+df['겹치는가중치'])&(df['인기일가중치']+df['겹치는가중치']<1.6), '트랜드'] = '트랜디'
df.loc[(1.6<=df['인기일가중치']+df['겹치는가중치'])&(df['인기일가중치']+df['겹치는가중치']<1.95), '트랜드'] = '쏘쏘'
df.loc[(1.95<=df['인기일가중치']+df['겹치는가중치'])&(df['인기일가중치']+df['겹치는가중치']<2.3), '트랜드'] = '낫트랜디'
df.loc[(2.3<=df['인기일가중치']+df['겹치는가중치']), '트랜드'] = '베리낫트랜디'


df_t['트랜드'] = np.nan
df_t.loc[(df_t['인기일가중치']+df_t['겹치는가중치']<1.25), '트랜드'] = '베리트랜디'
df_t.loc[(1.25<=df_t['인기일가중치']+df_t['겹치는가중치'])&(df_t['인기일가중치']+df_t['겹치는가중치']<1.6), '트랜드'] = '트랜디'
df_t.loc[(1.6<=df_t['인기일가중치']+df_t['겹치는가중치'])&(df_t['인기일가중치']+df_t['겹치는가중치']<1.95), '트랜드'] = '쏘쏘'
df_t.loc[(1.95<=df_t['인기일가중치']+df_t['겹치는가중치'])&(df_t['인기일가중치']+df_t['겹치는가중치']<2.3), '트랜드'] = '낫트랜디'
df_t.loc[(2.3<=df_t['인기일가중치']+df_t['겹치는가중치']), '트랜드'] = '베리낫트랜디'

In [58]:
# 금액점수

df['금액점수'] = (df['완전낮은금액비율'].round(4)*0.5)+(df['낮은금액비율'].round(4)*1)+(df['보통금액비율'].round(4)*1.5)+(df['높은금액비율'].round(4)*2)+(df['완전높은금액비율'].round(4)*2.5)

df_t['금액점수'] = (df_t['완전낮은금액비율'].round(4)*0.5)+(df_t['낮은금액비율'].round(4)*1)+(df_t['보통금액비율'].round(4)*1.5)+(df_t['높은금액비율'].round(4)*2)+(df_t['완전높은금액비율'].round(4)*2.5)

In [59]:
# 실구매범위등급

np.percentile(df['실구매값범위'], [0,20,40,60,80,100], interpolation='nearest')
df['실구매범위등급'] = np.nan
df.loc[df['실구매값범위']<=12700, '실구매범위등급'] = '매우좁음'
df.loc[(12700<df['실구매값범위'])&(df['실구매값범위']<=239700), '실구매범위등급'] = '좁음'
df.loc[(239700<df['실구매값범위'])&(df['실구매값범위']<=393200), '실구매범위등급'] = '보통'
df.loc[(393200<df['실구매값범위'])&(df['실구매값범위']<=711500), '실구매범위등급'] = '넓음'
df.loc[(711500<df['실구매값범위']), '실구매범위등급'] = '매우넓음'

np.percentile(df_t['실구매값범위'], [0,20,40,60,80,100], interpolation='nearest')
df_t['실구매범위등급'] = np.nan
df_t.loc[df_t['실구매값범위']<=125200, '실구매범위등급'] = '매우좁음'
df_t.loc[(125200<df_t['실구매값범위'])&(df_t['실구매값범위']<=246050), '실구매범위등급'] = '좁음'
df_t.loc[(246050<df_t['실구매값범위'])&(df_t['실구매값범위']<=402800), '실구매범위등급'] = '보통'
df_t.loc[(402800<df_t['실구매값범위'])&(df_t['실구매값범위']<=730460), '실구매범위등급'] = '넓음'
df_t.loc[(730460<df_t['실구매값범위']), '실구매범위등급'] = '매우넓음'

In [60]:
# 가격유동점수
df['가격유동점수'] = np.nan
df.loc[(df['금액점수']<=1)&(df['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'S'
df.loc[(1<df['금액점수'])&(df['금액점수']<=1.5)&(df['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'I'
df.loc[(1.5<df['금액점수'])&(df['금액점수']<=2)&(df['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'A'
df.loc[(2<df['금액점수'])&(df['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'VVIP'

df.loc[(df['금액점수']<=1)&(df['실구매범위등급']=='좁음'), '가격유동점수'] = 'R'
df.loc[(1<df['금액점수'])&(df['금액점수']<=1.5)&(df['실구매범위등급']=='좁음'), '가격유동점수'] = 'M'
df.loc[(1.5<df['금액점수'])&(df['금액점수']<=2)&(df['실구매범위등급']=='좁음'), '가격유동점수'] = 'B'
df.loc[(2<df['금액점수'])&(df['실구매범위등급']=='좁음'), '가격유동점수'] = 'VIP'

df.loc[(df['금액점수']<=1)&(df['실구매범위등급']=='보통'), '가격유동점수'] = 'Q'
df.loc[(1<df['금액점수'])&(df['금액점수']<=1.5)&(df['실구매범위등급']=='보통'), '가격유동점수'] = 'L'
df.loc[(1.5<df['금액점수'])&(df['금액점수']<=2)&(df['실구매범위등급']=='보통'), '가격유동점수'] = 'H'
df.loc[(2<df['금액점수'])&(df['실구매범위등급']=='보통'), '가격유동점수'] = 'C'

df.loc[(df['금액점수']<=1)&(df['실구매범위등급']=='넓음'), '가격유동점수'] = 'P'
df.loc[(1<df['금액점수'])&(df['금액점수']<=1.5)&(df['실구매범위등급']=='넓음'), '가격유동점수'] = 'K'
df.loc[(1.5<df['금액점수'])&(df['금액점수']<=2)&(df['실구매범위등급']=='넓음'), '가격유동점수'] = 'G'
df.loc[(2<df['금액점수'])&(df['실구매범위등급']=='넓음'), '가격유동점수'] = 'D'

df.loc[(df['금액점수']<=1)&(df['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'O'
df.loc[(1<df['금액점수'])&(df['금액점수']<=1.5)&(df['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'J'
df.loc[(1.5<df['금액점수'])&(df['금액점수']<=2)&(df['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'F'
df.loc[(2<df['금액점수'])&(df['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'E'


df_t['가격유동점수'] = np.nan
df_t.loc[(df_t['금액점수']<=1)&(df_t['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'S'
df_t.loc[(1<df_t['금액점수'])&(df_t['금액점수']<=1.5)&(df_t['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'I'
df_t.loc[(1.5<df_t['금액점수'])&(df_t['금액점수']<=2)&(df_t['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'A'
df_t.loc[(2<df_t['금액점수'])&(df_t['실구매범위등급']=='매우좁음'), '가격유동점수'] = 'VVIP'

df_t.loc[(df_t['금액점수']<=1)&(df_t['실구매범위등급']=='좁음'), '가격유동점수'] = 'R'
df_t.loc[(1<df_t['금액점수'])&(df_t['금액점수']<=1.5)&(df_t['실구매범위등급']=='좁음'), '가격유동점수'] = 'M'
df_t.loc[(1.5<df_t['금액점수'])&(df_t['금액점수']<=2)&(df_t['실구매범위등급']=='좁음'), '가격유동점수'] = 'B'
df_t.loc[(2<df_t['금액점수'])&(df_t['실구매범위등급']=='좁음'), '가격유동점수'] = 'VIP'

df_t.loc[(df_t['금액점수']<=1)&(df_t['실구매범위등급']=='보통'), '가격유동점수'] = 'Q'
df_t.loc[(1<df_t['금액점수'])&(df_t['금액점수']<=1.5)&(df_t['실구매범위등급']=='보통'), '가격유동점수'] = 'L'
df_t.loc[(1.5<df_t['금액점수'])&(df_t['금액점수']<=2)&(df_t['실구매범위등급']=='보통'), '가격유동점수'] = 'H'
df_t.loc[(2<df_t['금액점수'])&(df_t['실구매범위등급']=='보통'), '가격유동점수'] = 'C'

df_t.loc[(df_t['금액점수']<=1)&(df_t['실구매범위등급']=='넓음'), '가격유동점수'] = 'P'
df_t.loc[(1<df_t['금액점수'])&(df_t['금액점수']<=1.5)&(df_t['실구매범위등급']=='넓음'), '가격유동점수'] = 'K'
df_t.loc[(1.5<df_t['금액점수'])&(df_t['금액점수']<=2)&(df_t['실구매범위등급']=='넓음'), '가격유동점수'] = 'G'
df_t.loc[(2<df_t['금액점수'])&(df_t['실구매범위등급']=='넓음'), '가격유동점수'] = 'D'

df_t.loc[(df_t['금액점수']<=1)&(df_t['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'O'
df_t.loc[(1<df_t['금액점수'])&(df_t['금액점수']<=1.5)&(df_t['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'J'
df_t.loc[(1.5<df_t['금액점수'])&(df_t['금액점수']<=2)&(df_t['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'F'
df_t.loc[(2<df_t['금액점수'])&(df_t['실구매범위등급']=='매우넓음'), '가격유동점수'] = 'E'

In [61]:
# 브랜드 다양성

f = df_train.groupby('custid')['brd_nm'].agg([('브랜드다양성','nunique')]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

f_t = df_test.groupby('custid')['brd_nm'].agg([('브랜드다양성','nunique')]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [62]:
# 요일 다양성

f = df_train.groupby('custid')['sales_dayofweek'].agg([('요일다양성','nunique')]).reset_index()
df = df.merge(f.iloc[:,[0,-1]], how='left',on='custid')

f_t = df_test.groupby('custid')['sales_dayofweek'].agg([('요일다양성','nunique')]).reset_index()
df_t = df_t.merge(f_t.iloc[:,[0,-1]], how='left',on='custid')

In [63]:
# 환불상품구매건수

f = df_train.query('net_amt<0').groupby('custid')['goodcd'].agg([
    ('환불건수','count')]).reset_index()
df = df.merge(f, how='left', on='custid')

f_t = df_test.query('net_amt<0').groupby('custid')['goodcd'].agg([
    ('환불건수','count')]).reset_index()
df_t = df_t.merge(f, how='left', on='custid')

df = df.fillna(0)
df_t = df_t.fillna(0)

In [ ]:
# 수입, 할부

In [68]:
len(df_train.query('inst_mon>1').groupby('custid')), len(df_train.groupby('custid'))

(19233, 21587)

In [69]:
df_train.columns

Index(['custid', 'sales_dayofweek', 'str_nm', 'goodcd', 'brd_nm', 'corner_nm',
       'pc_nm', 'part_nm', 'team_nm', 'buyer_nm', 'import_flg', 'tot_amt',
       'dis_amt', 'net_amt', 'inst_mon', 'inst_fee', 'dt', 'city', 'refund',
       'dis/tot', '완전대분류', '가격', '계절', '분기'],
      dtype='object')

In [122]:
df.to_csv('피쳐_train.csv')
df_t.to_csv('피쳐_test.csv')

In [ ]:
# 중복구매상품이랑 주상품군이랑 겹치는게 많으니까 하나는 지우고,
# 주구매품목part넣을거면 예주가만든 part_nm은 넣지 말고, 
# 주상품군, 주구매품목part, 주구매품목완전, 주구매브랜드, 주구매품목buy, 주환불상품, 중복구매브랜드, 중복구매상품 - PCA

In [102]:
df_train[['brd_nm','city']]

,brd_nm,city
0,에스티로더,강남구
1,시슬리,강남구
2,크리니크,강남구
3,듀퐁,강남구
4,랑콤,강남구
...,...,...
625079,바닐라,강남구
625080,랑콤,강남구
625081,루이까또즈,강동구
625082,소피에르핀,강동구


In [3]:
df_all = pd.concat([df_train, df_test])

In [85]:
a = pd.pivot_table(df_all, index='custid', columns='brd_nm', values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
b = pd.pivot_table(df_all, index='custid', columns='brd_nm', values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])
a.to_csv('brd_nm원핫.csv')
b.to_csv('brd_nm원핫_test.csv')

In [121]:
c = pd.pivot_table(df_all, index='custid', columns='city', values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
d = pd.pivot_table(df_all, index='custid', columns='city', values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])
c.to_csv('city원핫.csv')
d.to_csv('city원핫_test.csv')

In [69]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1040039 entries, 0 to 414954
Data columns (total 25 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   custid           1040039 non-null  int64         
 1   sales_dayofweek  1040039 non-null  object        
 2   str_nm           1040039 non-null  object        
 3   goodcd           1040039 non-null  int64         
 4   brd_nm           1040039 non-null  object        
 5   corner_nm        1040039 non-null  object        
 6   pc_nm            1040039 non-null  object        
 7   part_nm          1040039 non-null  object        
 8   team_nm          1040039 non-null  object        
 9   buyer_nm         1040039 non-null  object        
 10  import_flg       1040039 non-null  int64         
 11  tot_amt          1040039 non-null  int64         
 12  dis_amt          1040039 non-null  int64         
 13  net_amt          1040039 non-null  int64         
 14  ins

In [74]:
df_all['month'] = df_all['dt'].dt.month
e = pd.pivot_table(df_all, index='custid', columns='month', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
e.columns = ['1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월']
f = pd.pivot_table(df_all, index='custid', columns='month', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])
f.columns = ['1월','2월','3월','4월','5월','6월','7월','8월','9월','10월','11월','12월']
e.to_csv('월별실구매액합.csv')
f.to_csv('월별실구매액합_test.csv')

In [76]:
g = pd.pivot_table(df_all, index='custid', columns='sales_dayofweek', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
h = pd.pivot_table(df_all, index='custid', columns='sales_dayofweek', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])

g.to_csv('요일별실구매액합.csv')
h.to_csv('요일별실구매액합_test.csv')

In [78]:
i = pd.pivot_table(df_all, index='custid', columns='계절', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
j = pd.pivot_table(df_all, index='custid', columns='계절', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])

i.to_csv('계절별실구매액합.csv')
j.to_csv('계절별실구매액합_test.csv')

In [81]:
k = pd.pivot_table(df_all, index='custid', columns='str_nm', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
l = pd.pivot_table(df_all, index='custid', columns='str_nm', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])

k.to_csv('지점별실구매액합.csv')
l.to_csv('지점별실구매액합_test.csv')

In [86]:
m = pd.pivot_table(df_all, index='custid', columns='part_nm', values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest'). \
                         drop(columns=['custid'])
n = pd.pivot_table(df_all, index='custid', columns='part_nm', values='tot_amt',
                         aggfunc=lambda x: np.where(len(x) >=1, 1, 0), fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest'). \
                         drop(columns=['custid'])
m.to_csv('part_nm원핫.csv')
n.to_csv('part_nm원핫_test.csv')

In [7]:
o = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='corner_nm', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
p = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='corner_nm', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest')

o.to_csv('corner별실구매액합.csv')
p.to_csv('corner별실구매액합_test.csv')

In [8]:
q = pd.pivot_table(df_all.query('net_amt<0'), index='custid', columns='corner_nm', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
r = pd.pivot_table(df_all.query('net_amt<0'), index='custid', columns='corner_nm', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest')

q.to_csv('corner별환불합.csv')
r.to_csv('corner별환불합_test.csv')

In [10]:
s = pd.pivot_table(df_all.query('net_amt<0'), index='custid', columns='str_nm', values='tot_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
s.columns = ['custid','tot무역점','tot본점','tot신촌점','tot천호점']

t = pd.pivot_table(df_all.query('net_amt<0'), index='custid', columns='str_nm', values='tot_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest')
t.columns = ['custid','tot무역점','tot본점','tot신촌점','tot천호점']

s.to_csv('지점별tot환불합.csv')
t.to_csv('지점별tot환불합_test.csv')

In [11]:
u = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='pc_nm', values='inst_mon',
                         aggfunc=np.mean, fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
v = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='pc_nm', values='inst_mon',
                         aggfunc=np.mean, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest')
u.to_csv('pc별할부평균.csv')
v.to_csv('pc별할부평균_test.csv')

In [12]:
w = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='part_nm', values='import_flg',
                         aggfunc=sum, fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
x = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='part_nm', values='import_flg',
                         aggfunc=sum, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest')
w.to_csv('part별수입여부합.csv')
x.to_csv('part별수입여부합_test.csv')

In [13]:
y = pd.pivot_table(df_all.query('net_amt<0'), index='custid', columns='sales_dayofweek', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
y.columns = ['custid','금요일','목요일','수요일','월요일','일요일','토요일','화요일']
z = pd.pivot_table(df_all.query('net_amt<0'), index='custid', columns='sales_dayofweek', values='net_amt',
                         aggfunc='sum', fill_value=0).\
                         reset_index(). \
                         query('custid in @IDtest')
z.columns = ['custid','금요일','목요일','수요일','월요일','일요일','토요일','화요일']

y.to_csv('요일별환불합.csv')
z.to_csv('요일별환불합_test.csv')

In [76]:
aa = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='sales_dayofweek', values='inst_mon',
                         aggfunc=np.mean, fill_value=0).\
                         reset_index(). \
                         query('custid not in @IDtest')
aa.columns = ['custid','할부금요일','할부목요일','할부수요일','할부월요일','할부일요일','할부토요일','할부화요일']
bb = pd.pivot_table(df_all.query('net_amt>0'), index='custid', columns='sales_dayofweek', values='inst_mon',
                         aggfunc=np.mean, fill_value=0). \
                         reset_index(). \
                         query('custid in @IDtest')
bb.columns = ['custid','할부금요일','할부목요일','할부수요일','할부월요일','할부일요일','할부토요일','할부화요일']
aa.to_csv('요일별할부평균.csv')
bb.to_csv('요일별할부평균_test.csv')

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0
